In [83]:
from abc import ABC, abstractmethod
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C as sb_A2C
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import gym
from datetime import datetime
from sys import path
from os.path import dirname as dir

path.append(dir(path[0]))
print(path)
#__package__ = "examples"

from FinancialEnvLayer.datacollector import CustomDatasetImporter
from FinancialEnvLayer.datacollector import DataDownloader
from FinancialEnvLayer.dataprocessor import FeatureEngineer

['/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi/AgentLayer', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/site-packages/selenium-3.141.0-py3.8.egg', '/usr/local/lib/python3.8/site-packages/urllib3-1.26.4-py3.8.egg', '/Users/doganparlak/Library/Python/3.8/lib/python/site-packages', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi']


In [6]:
class Environment(gym.Env, ABC):

    @abstractmethod
    def reset(self):
        pass

    @abstractmethod
    def step(self, action):
        pass

    @abstractmethod
    def render(self, mode="human"):
        pass

    @abstractmethod
    def get_env(self):
        pass

    @staticmethod
    def softmax_normalization(actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output


In [8]:
class Agent(ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass
    
    @abstractmethod
    def load_model():
        pass

In [9]:
class ConventionalAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass
    
    @abstractmethod
    def _return_predict():
        pass

    @abstractmethod
    def _weight_optimization():
        pass

In [10]:
class RLAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass

In [79]:
class PortfolioEnv(Environment):

    def __init__(self,
                 df: pd.DataFrame,  # input data
                 port_dim: int,  # number of unique securities in the investment universe
                 hmax: float,  # maximum number of shares to trade
                 initial_amount: float,  # initial cash value
                 transaction_cost_pct: float,  # transaction cost percentage per trade
                 reward_scaling: float,  # scaling factor for reward as training progresses
                 state_space: int,  # the dimension of input features (state space)
                 action_space: int,  # number of actions, which is equal to portfolio dimension
                 tech_indicator_list: list,  # a list of technical indicator names
                 turbulence_threshold=None,  # a threshold to control risk aversion
                 lookback=252,  #
                 day=0):  # an increment number to control date

        self.df = df
        self.day = day
        self.lookback = lookback
        self.port_dim = port_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct = transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1, shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                            shape=(self.state_space + len(self.tech_indicator_list), self.state_space))
        

        # load data from a pandas dataframe
        print(self.df.head(1))
        self.data = self.df.loc[self.day, :]
        print(self.data.head(1))
        self.covs = self.data['cov_list'][0]
        #print(self.data[tech])
        self.state = np.append(np.array(self.covs), [self.data[tech] for tech in self.tech_indicator_list],
                            axis=0)
        self.terminal = False
        #self.turbulence_threshold = turbulence_threshold
        # initalize state: initial portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory = [[1 / self.port_dim] * self.port_dim]
        self.date_memory = [self.data["date"]]

    def reset(self):
        pass

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(), 'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory, 'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / \
                         df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal, {}

        else:
            weights = Environment.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            # load next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self.covs = self.data['cov_list'][0]
            self.state = np.append(np.array(self.covs), [self.data[tech] for tech in self.tech_indicator_list],
                                   axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data["date"].unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, {}

    def render(self, mode='human'):
        return self.state

    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        # print(len(date_list))
        # print(len(asset_list))
        df_account_value = pd.DataFrame({'date': date_list, 'daily_return': portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [11]:
class A2C(RLAgent):

    def __init__(self,
                 policy="MlpPolicy",
                 env=None,
                 learning_rate: float = 7e-4,
                 n_steps: int = 5,
                 gamma: float = 0.99,
                 gae_lambda: float = 1.0,
                 ent_coef: float = 0.0,
                 vf_coef: float = 0.5,
                 max_grad_norm: float = 0.5,
                 rms_prop_eps: float = 1e-5,
                 use_rms_prop: bool = True,
                 use_sde: bool = False,
                 sde_sample_freq: int = -1,
                 normalize_advantage: bool = False,
                 tensorboard_log=None,
                 create_eval_env: bool = False,
                 policy_kwargs=None,
                 verbose: int = 0,
                 seed=None,
                 device="auto",
                 _init_setup_model: bool = True):

        self.env = env
        # self.model = A2C(model_params["policy"], model_params["environment"], model_params["verbose"])
        self.model = sb_A2C(policy=policy,
                            env=self.env,
                            tensorboard_log=tensorboard_log,
                            verbose=verbose,
                            policy_kwargs=policy_kwargs,
                            seed=seed,
                            **model_kwargs)

    def train_model(self, **train_params):
        self.model = self.model.learn(total_timesteps=train_params["total_timesteps"],
                                      log_interval=train_params["log_interval"])
        return self.model

    def predict(self, **test_params):

        test_env, test_obs = test_params["environment"].environment()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        test_env.reset()
        for i in range(len(test_params["environment"].df.index.unique())):
            action, _states = self.model.predict(test_obs, deterministic=test_params["deterministic"])
            test_obs, rewards, dones, info = test_env.step(action)
            if i == (len(test_params["environment"].df.index.unique()) - 2):
                account_memory = test_env.env_method(method_name="save_asset_memory")
                actions_memory = test_env.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]

    def save_model(self, path):
        self.model.save(path)

    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model

In [12]:
TECHNICAL_INDICATORS_LIST = [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
]
env_kwargs = {
    "hmax": 100, #maximum number of shares to trade
    "initial_amount": 1000000, # initial cash
    "transaction_cost_pct": 0.001, # transaction cost percentage
    "state_space": 29, # number of unique stocks 
    "stock_dim": 29, # number of unique stocks
    "tech_indicator_list": TECHNICAL_INDICATORS_LIST, # technical indicators
    "action_space": 29, # number of stocks in training data
    "reward_scaling": 1e-1  #hyperparameter
}

In [32]:
#Gather user parameters
with open("../user_params.yaml", "r") as stream:
    try:
        user_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

tickers = user_params["tickers"]
env_kwargs = user_params["env_params"]

print("\nTest 3: Downloading from Yahoo.........")
downloaded_df = DataDownloader.download_data(start_date='2009-01-01',
                                                end_date='2021-10-31',
                                                ticker_list=tickers)
print(downloaded_df.head())



Test 3: Downloading from Yahoo.........
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10354, 8)
         date       open       high        low      close     volume   tic  \
0  2008-12-31   3.070357   3.133571   3.047857   2.610107  607541200  AAPL   
1  2008-12-31  50.740002  51.689999  49.910000  51.279999    7792200  AMZN   
2  2008-12-31  82.239998  86.150002  81.120003  69.224129   14894100    GS   
3  2009-01-02   3.067143   3.251429   3.041429   2.775245  746015200  AAPL   
4  2009-01-02  51.349998  54.529999  51.070000  54.360001    7296400  AMZN   

   day  
0    2  
1    2  
2    2  
3    4  
4    4  


In [35]:
print("\nTest 4: Feature engineer.........")
df_processed = FeatureEngineer.add_features(df=downloaded_df,
                                            use_default=True,
                                            tech_indicator_list=TECHNICAL_INDICATORS_LIST,
                                            use_vix=True,
                                            use_turbulence=True,
                                            user_defined_feature=True)  # included technical indicators as features

print(df_processed.head())


Test 4: Feature engineer.........
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix
Successfully added turbulence index
Successfully added user defined features
Successfully added covariances
        date   tic        open        high         low       close  \
0 2009-12-31  AAPL    7.611786    7.619643    7.520000    6.444381   
1 2009-12-31  AMZN  137.089996  137.279999  134.520004  134.520004   
2 2009-12-31    GS  167.289993  170.130005  166.929993  140.212418   
3 2010-01-04  AAPL    7.622500    7.660714    7.585000    6.544687   
4 2010-01-04  AMZN  136.250000  136.610001  133.139999  133.899994   

        volume  day      macd     boll_ub  ...     rsi_30      cci_30  \
0  352410800.0  3.0  0.105383    6.541181  ...  60.410575  155.820993   
1    4523000.0  3.0  2.339104  142.484494  ...  58.842982   24.419618   
2    6401800.0  3.0 -1.015380  140.049410  ...  49.863

In [80]:
env = PortfolioEnv(df=df_processed, **env_kwargs) #train parametresi training data olucak (dataframe) daha koyulmadi 
env_train, _ = env.get_env()

        date   tic      open      high   low     close       volume  day  \
0 2009-12-31  AAPL  7.611786  7.619643  7.52  6.444381  352410800.0  3.0   

       macd   boll_ub  ...     rsi_30      cci_30      dx_30  close_30_sma  \
0  0.105383  6.541181  ...  60.410575  155.820993  31.312031      6.114613   

   close_60_sma    vix  turbulence  daily_return  \
0      6.057069  21.68         0.0     -0.953448   

                                            cov_list  \
0  [[0.0004566258236396403, 0.0002820014596461389...   

                                         return_list  
0  tic             AAPL      AMZN        GS
date ...  

[1 rows x 21 columns]
date    2009-12-31 00:00:00
Name: 0, dtype: object


In [91]:
env_train.action_space

In [86]:
a2c_params = {"policy" : "MlpPolicy", 
                "environment" : env_train,
                "verbose" : 1}

train_params = {"total_timesteps": 25000,
                "log_interval": 100}

test_params = {"environment" : env_train,
               "deterministic" : True}

In [87]:
#object creation
a2c = A2C(env = env_train, policy = "MlpPolicy")

AssertionError: The algorithm only supports (<class 'gym.spaces.box.Box'>, <class 'gym.spaces.discrete.Discrete'>, <class 'gym.spaces.multi_discrete.MultiDiscrete'>, <class 'gym.spaces.multi_binary.MultiBinary'>) as action spaces but None was provided

In [ ]:
#training
a2c.train_model(train_params)

In [ ]:
#predicting
a2c.predict(test_params)

In [ ]:
#saving 
a2c.save_model("a2c_model")

In [ ]:
#loading
loaded_a2c_model = a2c.load_model("a2c_model")